In [ ]:
 # OBJECTIVES -----------------------------------------------------------------------------------------------------------
    def MinPenalty(m): 
        penalty_sum = sum([m.ground_penalty[n,c,gc] for n in range(1,N+1) for c in contacts for gc in ground_constraints])
        return penalty_sum 
    m.MinPenalty = Objective(rule = MinPenalty)
    
    
    def MinTorque(m):
        torque_sum = 0 
        for n in range(1,N+1):
            for j in joints:
                torque_sum += m.force_a[n, j,'R']**2
        return torque_sum
    m.MinTorque = Objective(rule = MinTorque)
    
    m.MinTorque.deactivate()

In [ ]:
def init_opt(): # initializes solver
    opt = SolverFactory('ipopt',executable = '/opt/homebrew/bin/ipopt')
    opt.options["linear_solver"] = 'mumps' #'ma97'
    opt.options["print_level"] = 3 # prints a log with each iteration (you want to this - it's the only way to see progress.)
    opt.options["max_iter"] = 30000 # maximum number of iterations
    opt.options["max_cpu_time"] = 3600 # maximum cpu time in seconds
    opt.options["Tol"] = 1e-6 # the tolerance for feasibility. Considers constraints satisfied when they're within this margin.
    
    opt.options["OF_acceptable_obj_change_tol"] = 1e-4
    opt.options["OF_ma86_scaling"] = 'none'
    
    return opt

In [ ]:
def high_drop(m):
    for dof in DOFs:
        m.dq[1,dof].fix(0)  #set rate of change of all DOFs to 0 at N=1
        m.q[1,dof].fix(0)   #set value of all DOFs to 0 at N=1
    m.q[1,'y','b'].fix(5)   #set y value of body link to 5 at N=1
    m.q[1,'tht','R'].fix(3*np.pi/2)
    
    for n in range(1,N+1):
        for j in joints:
            for s in sides:
                m.force_a[n,j,s].fix(0) #set actuator forces of all joints to 0 for all N
                
    for n in range(1,N+1):
        for c in contacts:
            m.GRF[n,c,'y'].fix(0) #set y component of ground reaction force of contact points to 0 for all N
    return m

In [ ]:
def check_model(m,result):
    if result.solver.status != SolverStatus.ok:
        print('fail: solver not ok')
        return 0
    if result.solver.termination_condition != TerminationCondition.optimal:
        print('fail: did not find optimal solution')
        return 0
    max_ground_penalty = np.max([m.ground_penalty[n,c,gc].value for n in range(1,N+1) for c in contacts for gc in ground_constraints])
    thr = 1e-3
    if max_ground_penalty > thr:
        print('fail: ground penalty violated')
        return 0
    print('check_model: success')
    return 1

In [ ]:
#Deactivate first objective, activate second objective
def activate_objective(m):
    m.MinPenalty.deactivate()
    m.MinTorque.activate()
    for n in range(1,N+1):
        for c in contacts:
            for gc in ground_constraints:
                m.ground_penalty[n,c,gc].setub(1e-4)
    return m

In [ ]:
from time import time
def solve_model(m,filename):
    opt = init_opt()
    success = 0
    try:
        result = opt.solve(m,tee= True)
        success = check_model(m,result)
    except Exception as e:
        print('fail',e)
        return [success,m,0]
    
    if success > 0:
        #data = save_model(m,t2-t1,filename)
        print('solve_model: success')
        return [success,m,result]
    else:
        return [success,m,0]

In [ ]:
N = 100   #150 for 12.5m
hm = 0.02
mu = 0.95
sd = 1
#m = create_model(N,hm,mu)
#--------------------------------------- 2 stage optimisation ----------------------------------------------

m = create_model(N,hm,mu)

m = high_drop(m)



filename = r'biped_results/stop_highf_firstorder_i%ds%d_feasible'%(instance,sd)
    
# First Stage
print('Initialising Primary Objective function...')
success,m,result = solve_model(m,filename)

# Second Stage
if success > 0:
    m = activate_objective(m)
    filename = r'biped_results/stop_highf_firstorder_i%ds%d_optimal'%(instance,sd)
    print('Initialising second stage...')
    print('Initialising Secondary Objective function...')
    success,m,result = solve_model(m,filename)
